The 'starting_lineups_1.csv' file provied by Matthias at ASA is in wide format so it needs to be made tidy and vertical for ease of analysis and to be consistent with 'starting_lineups_2.csv' and 'starting_lineups_3.csv'. Below I use the melt() function in pandas to tidy the csv and then save a new csv for import to my postgres db.

In [36]:
import pandas as pd

In [37]:
df1 = pd.read_csv("starting_lineups_1.csv")
df1.head(10)

,gameID,team,home,formation,player1,player2,player3,player4,player5,player6,...,player9,player10,player11,bench1,bench2,bench3,bench4,bench5,bench6,bench7
0,914827,New York,1,4231,Luis Robles,Sal Zizzo,Connor Lade,Felipe,Ronald Zubar,Damien Perrinelle,...,Bradley Wright-Phillips,Sacha Kljestan,Mike Grella,Anthony Wallace,Kyle Reynish,Karl Ouimette,Sean Davis,Gonzalo Veron,Kemar Lawrence,Shaun Wright-Phillips
1,914827,Montreal,0,433,Evan Bush,Ambroise Oyongo,Donny Toia,Marco Donadel,Victor Cabrera,Laurent Ciman,...,Didier Drogba,Dominic Oduro,Dilly Duka,Hassoun Camara,Patrice Bernier,Andres Romero,Calum Mallace,Romario Williams,Eric Kronberg,Maxim Tissot
2,914828,Portland,1,442,Adam Larsen Kwarasey,Alvas Powell,Jorge Villafana,Diego Chara,Nat Borchers,Liam Ridgewell,...,Fanendo Adi,Maximiliano Urruti,Rodney Wallace,Gaston Fernandez,Dairon Asprilla,Jack Jewsbury,Jeanderson,Taylor Peay,Norberto Paparatto,Andrew Weber
3,914828,FC Dallas,0,4231,Chris Seitz,Atiba Harris,Zach Loyd,Michel,Walker Zimmerman,Matt Hedges,...,David Texeira,Tesho Akindele,Fabian Castillo,Moises Hernandez,Kyle Bekker,Blas Perez,Michael Barrios,Je-Vaughn Watson,Alejandro Zendejas,Jesse Gonzalez
4,914829,New York,1,4231,Luis Robles,Chris Duvall,Kemar Lawrence,Felipe,Matt Miazga,Damien Perrinelle,...,Bradley Wright-Phillips,Sacha Kljestan,Mike Grella,Connor Lade,Anatole Bertrand Abang,Sal Zizzo,Kyle Reynish,Karl Ouimette,Sean Davis,Dane Richards
5,914829,San Jose,0,433,David Bingham,Marvell Wynne,Cordell Cato,Fatai Alashe,Victor Bernardez,Clarence Goodson,...,Innocent,Sanna Nyassi,Shea Salinas,Ty Harden,Chris Wondolowski,Bryan Meredith,Michael Fucito,Tommy Thompson,Leandro Barrera,Paulo Renato
6,914830,DC United,1,442,Andrew Dykstra,Sean Franklin,Taylor Kemp,Davy Arnaud,Bobby Boswell,Kofi Opare,...,Chris Pontius,Jairo Arrieta,Chris Rolfe,Luis Silva,Nick DeLeon,Miguel Aguilar,Chris Korb,Jared Jeffrey,Conor Doyle,Travis Worra
7,914830,Houston,0,4231,Tyler Deric,Kofi Sarkodie,DaMarcus Beasley,Nathan Sturgis,Raul Rodriguez,Jermaine Taylor,...,Giles Barnes,Ricardo Clark,Brad Davis,Alex,Rob Lovejoy,Will Bruin,Chandler Hoffman,David Horst,Joe Willis,Leonel Miranda
8,914831,Columbus,1,4231,Steve Clark,Hernan Grana,Waylon Francis,Tony Tchani,Michael Parkhurst,Emanuel Pogatetz,...,Kei Kamara,Federico Higuain,Justin Meram,Tyson Wahl,Chris Klute,Kristinn Steindorsson,Kevan George,Hector Jimenez,Aaron Schoenfeld,Matt Lampson
9,914831,Orlando City,0,433,Donovan Ricketts,Rafael Ramos,Brek Shea,Amobi Okugo,Aurelien Collin,Seb Hines,...,Cyle Larin,Kevin Molino,Kaka,Tyler Turner,Harrison Heath,Carlos Rivas,Eric Avila,Luke Boden,Bryan Rochez,Earl Edwards


This csv is in the _tidy_ format we need the data in.

In [42]:
df2 = pd.read_csv("starting_lineups_2_c.csv")
df2.head(10)

,gameID,team,home,formation,position,players
0,1041727,New York,1,4231,player1,Luis Robles
1,1041727,New York,1,4231,player2,Sal Zizzo
2,1041727,New York,1,4231,player3,Kemar Lawrence
3,1041727,New York,1,4231,player4,Dax McCarty
4,1041727,New York,1,4231,player5,Ronald Zubar
5,1041727,New York,1,4231,player6,Gideon Baah
6,1041727,New York,1,4231,player7,Lloyd Sam
7,1041727,New York,1,4231,player8,Felipe
8,1041727,New York,1,4231,player9,Bradley Wright-Phillips
9,1041727,New York,1,4231,player10,Sacha Kljestan


In [43]:
# let's try to unpivot by melting df1
# make a list object with all columns to be melted into single column named 'players'

melt_list = []
for i in range(1,12):
    melt_list.append('player'+str(i))
for i in range(1, 8):
    melt_list.append('bench'+str(i))
    
print melt_list


['player1', 'player2', 'player3', 'player4', 'player5', 'player6', 'player7', 'player8', 'player9', 'player10', 'player11', 'bench1', 'bench2', 'bench3', 'bench4', 'bench5', 'bench6', 'bench7']


In [44]:
tidy = pd.melt(df1.reset_index(),
               id_vars = ['gameID', 'team', 'home', 'formation'], 
               value_vars = melt_list,
               var_name = 'position',
               value_name='players')

In [45]:
tidy.head()

,gameID,team,home,formation,position,players
0,914827,New York,1,4231,player1,Luis Robles
1,914827,Montreal,0,433,player1,Evan Bush
2,914828,Portland,1,442,player1,Adam Larsen Kwarasey
3,914828,FC Dallas,0,4231,player1,Chris Seitz
4,914829,New York,1,4231,player1,Luis Robles


In [46]:
#resort by gameID
tidy = tidy.sort_values(['gameID', 'home', 'position']).reset_index(drop = True)
tidy.head()

,gameID,team,home,formation,position,players
0,908236,Chicago,0,41212,bench1,Matt Polster
1,908236,Chicago,0,41212,bench2,Adailton
2,908236,Chicago,0,41212,bench3,Alex
3,908236,Chicago,0,41212,bench4,Greg Cochrane
4,908236,Chicago,0,41212,bench5,Jon Busch
5,908236,Chicago,0,41212,bench6,Guly
6,908236,Chicago,0,41212,bench7,Michael Stephens
7,908236,Chicago,0,41212,player1,Sean Johnson
8,908236,Chicago,0,41212,player10,Quincy Amarikwa
9,908236,Chicago,0,41212,player11,Harry Shipp


In [48]:
# export tidied df1 as csv for import to db

tidy.to_csv('starting_lineups_1_c.csv', index = False)